In [1]:
from imagery_psychophysics.src.variational import *
from imagery_psychophysics.src.stirling_maps import stirling_num_of_2nd_kind as snk
from imagery_psychophysics.src.stirling_maps import stirling_partitions
from skimage.morphology import binary_dilation, binary_erosion
import numpy as np
import pandas as pd
import math
import copy
from matplotlib import pyplot as plt
from os.path import join
from PIL.Image import open as open_image
from mpl_toolkits.axes_grid1 import ImageGrid
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib inline

Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
/tmp/try_flags_WTa9w0.c:4:19: fatal error: cudnn.h: No such file or directory
compilation terminated.

Mapped name None to device cuda: GeForce GTX TITAN X (0000:01:00.0)


In [2]:
resultsDict = dict()
experiment_df = open_imagery_probe_data()

In [9]:
experiment_df['image'].unique()

array(['candle_01', 'peaches_05', 'cheeseburger_03', 'grape_juice_02'],
      dtype=object)

In [10]:
for subject, subject_group in experiment_df.groupby('subj'):
    resultsDict[subject] = dict()
    for state, state_group in subject_group.groupby('state'):
        resultsDict[subject][state] = dict()
        for targetImageName, target_group in state_group.groupby('image'):
            ##get data
            windows, resp, _, targetObjectMap,targetImage = open_imagery_probe_data(subject, state, targetImageName)
            targetObjectMap = targetObjectMap.resize(targetImage.size) ##have to do this because not matched in size
            
            ##==========construct model random variables
            ##number of objects
            nObj = numObjects()

            ##dispersion on category prior: here we set a hyperprior
            pDisp = priorDispersion()
            dispersion = 1.0
            pDisp.set_value(dispersion)

            ##resolution of object map Z
            nPixels = numPixels()

            ##category prior and object map
            catProb = categoryProbs(nObj,pDisp)
            Z = latentObjMap(catProb,nPixels)

            ##noise params
            nP = noiseParams()

            ##windows: we change their shape a little to make them easier to work with
            desiredWindowShape = (375,600)
            workingScale = 5
            w = probes()
            resolutions, workingResolution = w.resolve(desiredWindowShape, workingScale)
            w.set_value(w.reshape(windows, workingResolution),flatten=True)
            print 'working resolution is (%d, %d)' %(workingResolution[0], workingResolution[1])


            ##response object
            r = responses(Z,nP)

            ##fake data
            r.set_values(windows=w)

            r.set_values(data=resp)

            print 'total observations: %d' %(r.N)

            tMap = target_image()
            targetObjectMap_test, targetImage_test = tMap.reshape(targetObjectMap,workingResolution,targetImage=targetImage)
            tMap.set_values(targetObjectMap_test, targetImage_test)
            
            ##==gonna cheat here and set the number of objects using target map
            thisK = tMap.numberTargetObjects
            r.Z.categoryPrior.numObjects.set_value(tMap.numberTargetObjects)
            
            ##==fixing resolution at some reasonble thing (10 x 16)
            thisNumPix = (resolutions[1][0], resolutions[1][1])
            r.Z.numPixels.set_value(thisNumPix[0], thisNumPix[1])
            
            
            ##=============
            
            ##instantiate the variational inferences we want to perform
            iqZ = inferQZ()
            iqPi = inferQPi()

            ##...and the parameter optimizations (point-estimate) we want
            oNP = optimizeNoiseParams()

            ##variational inference combines them all together
            vi = VI(r, iqZ,oNP, iqPi)

            ### Run variational inference

            ##inference algorithm parameters
            initialNoisinessOfZ = 0.2
            pOn_init, pOff_init = .8, 0.2
            densityOfNoiseParamGrid = 50
            numStarterMaps = 20
            numSamplesForComputingObjectCountProbs = 4
            maxNumIterations = 50
            trainTestSplit = 0.8
            trainRegSplit = .8

            print '=========================(subject, state, target) = (%s, %s, %s) ====' %(subject, state, targetImageName)
            print '=========================(number of object = %d, number of pixels = (%d,%d) )' %(thisK, thisNumPix[0],thisNumPix[1])
            bestModel,_ = vi.run_VI(initialNoisinessOfZ, \
                                 pOn_init, pOff_init, \
                                 densityOfNoiseParamGrid, \
                                 numStarterMaps, \
                                 numSamplesForComputingObjectCountProbs, \
                                 maxNumIterations, \
                                 trainTestSplit, trainRegSplit, \
                                 optimizeHyperParams=False)

            resultsDict[subject][state][targetImageName]= copy.deepcopy(bestModel)

working resolution is (50, 80)
total observations: 345
=========================(subject, state, target) = (AS, img, candle_01) ====
=========================(number of object = 4, number of pixels = (10,16) )
!new best!
ELBO: -317.678467
goodness of fit: -412.779846
posterior_entropy: 95.101364
mean log of predictive distribution over test samples: -1.705736
percent correct over test samples: 34.545455


!new best!
ELBO: -114.191391
goodness of fit: -244.606918
posterior_entropy: 130.415527
mean log of predictive distribution over test samples: -1.055600
percent correct over test samples: 52.727273


!new best!
ELBO: -110.588226
goodness of fit: -234.757492
posterior_entropy: 124.169266
mean log of predictive distribution over test samples: -1.001981
percent correct over test samples: 60.000000


!new best!
ELBO: -97.302734
goodness of fit: -214.616928
posterior_entropy: 117.314194
mean log of predictive distribution over test samples: -0.966996
percent correct over test samples: 63.6

ELBO: -118.559341
goodness of fit: -132.735352
posterior_entropy: 14.176013
mean log of predictive distribution over test samples: -0.552938
percent correct over test samples: 80.000000


ELBO: -120.457268
goodness of fit: -134.343109
posterior_entropy: 13.885839
mean log of predictive distribution over test samples: -0.539441
percent correct over test samples: 80.000000


ELBO: -120.443192
goodness of fit: -134.081024
posterior_entropy: 13.637836
mean log of predictive distribution over test samples: -0.546384
percent correct over test samples: 80.000000


ELBO: -116.314217
goodness of fit: -131.629135
posterior_entropy: 15.314917
mean log of predictive distribution over test samples: -0.520589
percent correct over test samples: 81.818182


ELBO: -115.781509
goodness of fit: -130.696701
posterior_entropy: 14.915188
mean log of predictive distribution over test samples: -0.541721
percent correct over test samples: 78.181818


ELBO: -115.981415
goodness of fit: -131.114441
posterior_ent

ELBO: -103.863190
goodness of fit: -185.705521
posterior_entropy: 81.842331
mean log of predictive distribution over test samples: -0.987734
percent correct over test samples: 49.090909


ELBO: -107.720169
goodness of fit: -182.493439
posterior_entropy: 74.773270
mean log of predictive distribution over test samples: -1.087670
percent correct over test samples: 50.909091


ELBO: -107.368301
goodness of fit: -175.055862
posterior_entropy: 67.687561
mean log of predictive distribution over test samples: -0.966615
percent correct over test samples: 58.181818


ELBO: -113.536896
goodness of fit: -177.916275
posterior_entropy: 64.379379
mean log of predictive distribution over test samples: -1.103918
percent correct over test samples: 56.363636


ELBO: -115.881592
goodness of fit: -174.257782
posterior_entropy: 58.376190
mean log of predictive distribution over test samples: -1.039791
percent correct over test samples: 58.181818


ELBO: -115.241135
goodness of fit: -168.457809
posterior_ent

ELBO: -28.880043
goodness of fit: -38.985096
posterior_entropy: 10.105054
mean log of predictive distribution over test samples: -0.186087
percent correct over test samples: 96.363636


ELBO: -33.676544
goodness of fit: -44.196186
posterior_entropy: 10.519640
mean log of predictive distribution over test samples: -0.186087
percent correct over test samples: 96.363636


ELBO: -28.607765
goodness of fit: -38.985096
posterior_entropy: 10.377331
mean log of predictive distribution over test samples: -0.186087
percent correct over test samples: 96.363636


ELBO: -31.939648
goodness of fit: -41.590641
posterior_entropy: 9.650993
mean log of predictive distribution over test samples: -0.186087
percent correct over test samples: 96.363636


ELBO: -30.291500
goodness of fit: -38.985096
posterior_entropy: 8.693596
mean log of predictive distribution over test samples: -0.186087
percent correct over test samples: 96.363636


ELBO: -29.422974
goodness of fit: -38.985096
posterior_entropy: 9.562121

ELBO: -140.643677
goodness of fit: -262.560181
posterior_entropy: 121.916512
mean log of predictive distribution over test samples: -1.136788
percent correct over test samples: 65.454545


ELBO: -161.757950
goodness of fit: -282.620941
posterior_entropy: 120.862991
mean log of predictive distribution over test samples: -1.266188
percent correct over test samples: 58.181818


ELBO: -161.999359
goodness of fit: -271.264984
posterior_entropy: 109.265617
mean log of predictive distribution over test samples: -1.279129
percent correct over test samples: 52.727273


ELBO: -166.269318
goodness of fit: -266.930298
posterior_entropy: 100.660980
mean log of predictive distribution over test samples: -1.328091
percent correct over test samples: 38.181818


ELBO: -159.140625
goodness of fit: -258.298279
posterior_entropy: 99.157654
mean log of predictive distribution over test samples: -1.057231
percent correct over test samples: 61.818182


ELBO: -171.082184
goodness of fit: -263.108093
posterior

...

ELBO: -122.489349
goodness of fit: -149.745285
posterior_entropy: 27.255936
mean log of predictive distribution over test samples: -0.965436
percent correct over test samples: 65.454545


ELBO: -118.421669
goodness of fit: -146.419113
posterior_entropy: 27.997444
mean log of predictive distribution over test samples: -0.849956
percent correct over test samples: 65.454545


ELBO: -116.146309
goodness of fit: -141.993576
posterior_entropy: 25.847267
mean log of predictive distribution over test samples: -0.781948
percent correct over test samples: 72.727273


ELBO: -116.891670
goodness of fit: -142.703156
posterior_entropy: 25.811485
mean log of predictive distribution over test samples: -0.714133
percent correct over test samples: 74.545455


ELBO: -120.906219
goodness of fit: -146.478653
posterior_entropy: 25.572430
mean log of predictive distribution over test samples: -0.804842
percent correct over test samples: 72.727273


ELBO: -119.120079
goodness of fit: -142.658676
posterior_ent

ELBO: -87.931137
goodness of fit: -280.946289
posterior_entropy: 193.015152
mean log of predictive distribution over test samples: -1.216449
percent correct over test samples: 52.727273


ELBO: -93.021729
goodness of fit: -286.361511
posterior_entropy: 193.339783
mean log of predictive distribution over test samples: -1.214423
percent correct over test samples: 49.090909


ELBO: -86.714722
goodness of fit: -291.884430
posterior_entropy: 205.169708
mean log of predictive distribution over test samples: -1.296485
percent correct over test samples: 47.272727


ELBO: -86.354553
goodness of fit: -278.391357
posterior_entropy: 192.036804
mean log of predictive distribution over test samples: -1.213032
percent correct over test samples: 52.727273


ELBO: -94.788803
goodness of fit: -273.169373
posterior_entropy: 178.380569
mean log of predictive distribution over test samples: -1.199439
percent correct over test samples: 49.090909


ELBO: -98.498703
goodness of fit: -275.110626
posterior_entr

!new best!
ELBO: -102.385330
goodness of fit: -129.849396
posterior_entropy: 27.464066
mean log of predictive distribution over test samples: -1.042246
percent correct over test samples: 76.363636


!new best!
ELBO: -76.705246
goodness of fit: -94.938492
posterior_entropy: 18.233248
mean log of predictive distribution over test samples: -0.922502
percent correct over test samples: 80.000000


ELBO: -80.921173
goodness of fit: -94.603706
posterior_entropy: 13.682532
mean log of predictive distribution over test samples: -0.643347
percent correct over test samples: 80.000000


ELBO: -80.357483
goodness of fit: -92.005699
posterior_entropy: 11.648218
mean log of predictive distribution over test samples: -0.625118
percent correct over test samples: 80.000000


ELBO: -80.941216
goodness of fit: -92.005699
posterior_entropy: 11.064482
mean log of predictive distribution over test samples: -0.626336
percent correct over test samples: 80.000000


!new best!
ELBO: -67.905052
goodness of fit: -

!new best!
ELBO: -139.219940
goodness of fit: -294.033203
posterior_entropy: 154.813263
mean log of predictive distribution over test samples: -1.231889
percent correct over test samples: 58.181818


ELBO: -153.396942
goodness of fit: -286.875183
posterior_entropy: 133.478241
mean log of predictive distribution over test samples: -1.197905
percent correct over test samples: 56.363636


ELBO: -145.800842
goodness of fit: -282.517395
posterior_entropy: 136.716553
mean log of predictive distribution over test samples: -1.184139
percent correct over test samples: 50.909091


ELBO: -152.043671
goodness of fit: -287.632141
posterior_entropy: 135.588470
mean log of predictive distribution over test samples: -1.155444
percent correct over test samples: 58.181818


ELBO: -133.795258
goodness of fit: -279.053955
posterior_entropy: 145.258698
mean log of predictive distribution over test samples: -1.204720
percent correct over test samples: 56.363636


ELBO: -140.481049
goodness of fit: -280.6068

ELBO: -130.837463
goodness of fit: -194.437378
posterior_entropy: 63.599911
mean log of predictive distribution over test samples: -0.977785
percent correct over test samples: 56.363636


ELBO: -125.230103
goodness of fit: -190.842133
posterior_entropy: 65.612030
mean log of predictive distribution over test samples: -0.988848
percent correct over test samples: 56.363636


ELBO: -132.603943
goodness of fit: -195.257736
posterior_entropy: 62.653790
mean log of predictive distribution over test samples: -1.082239
percent correct over test samples: 54.545455


ELBO: -116.985756
goodness of fit: -180.319672
posterior_entropy: 63.333916
mean log of predictive distribution over test samples: -0.950088
percent correct over test samples: 56.363636


ELBO: -120.983734
goodness of fit: -184.975388
posterior_entropy: 63.991657
mean log of predictive distribution over test samples: -0.970033
percent correct over test samples: 58.181818




#### Store the results

In [11]:
modelList = []
for sbj,subjectDict in resultsDict.iteritems():
    for st, stateDict in subjectDict.iteritems():
        for targ, modelInstance in stateDict.iteritems():
            modelList.append([sbj,st,targ, modelInstance])

In [12]:
modelDf  = pd.DataFrame(data=modelList, columns=['subject', 'state', 'target', 'model'])
modelDf

,subject,state,target,model
0,AS,pcp,candle_01,<imagery_psychophysics.src.variational.VI obje...
1,AS,pcp,peaches_05,<imagery_psychophysics.src.variational.VI obje...
2,AS,img,candle_01,<imagery_psychophysics.src.variational.VI obje...
3,AS,img,peaches_05,<imagery_psychophysics.src.variational.VI obje...
4,z1,pcp,candle_01,<imagery_psychophysics.src.variational.VI obje...
5,z1,pcp,peaches_05,<imagery_psychophysics.src.variational.VI obje...
6,z1,img,candle_01,<imagery_psychophysics.src.variational.VI obje...
7,z1,img,peaches_05,<imagery_psychophysics.src.variational.VI obje...
8,KL,pcp,candle_01,<imagery_psychophysics.src.variational.VI obje...
9,KL,pcp,cheeseburger_03,<imagery_psychophysics.src.variational.VI obje...


In [13]:
store = pd.io.pytables.HDFStore('/home/tnaselar/FAST/imagery_psychophysics/multi_poly_probes/analysis/all_subjects.h5')

In [14]:
store['models'] = modelDf

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2882: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->['subject', 'state', 'target', 'model']]

  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
store.close()